In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
import os

In [3]:
# Define the path to your training and test data folders
train_data_dir = 'Training-Test-Data/asl_alphabet_train/asl_alphabet_train'
test_data_dir = 'Training-Test-Data/asl_alphabet_test/asl_alphabet_test'

# Specify the input image dimensions
image_width, image_height = 200, 200

# Assuming colored images
input_shape = (image_width, image_height, 3) 

In [4]:
# Set the number of classes based on the number of folders (labels) in the training data
num_classes = len(os.listdir(train_data_dir))

# Get the list of class folders
class_folders = [folder for folder in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, folder))]

# Set the number of classes based on the number of folders
num_classes = len(class_folders)

# Create an instance of the ImageDataGenerator for data augmentation and preprocessing
data_generator = ImageDataGenerator(rescale=1./255)

In [5]:
# Load and preprocess the training data
train_generator = data_generator.flow_from_directory(
    train_data_dir,
    target_size=(image_width, image_height),
    batch_size=32,
    classes=class_folders,
    class_mode='categorical'
)

# Load and preprocess the test data
test_generator = data_generator.flow_from_directory(
    test_data_dir,
    target_size=(image_width, image_height),
    batch_size=32,
    classes=class_folders,
    class_mode='categorical'
)

Found 87000 images belonging to 29 classes.
Found 28 images belonging to 29 classes.


In [6]:
# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))`
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 128)      0

In [11]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=5)

Epoch 1/5
2719/2719 [==============================] - 2052s 754ms/step - loss: 0.6633 - accuracy: 0.8028
Epoch 2/5
2719/2719 [==============================] - 2592s 953ms/step - loss: 0.0749 - accuracy: 0.9768
Epoch 3/5
2719/2719 [==============================] - 3984s 1s/step - loss: 0.0528 - accuracy: 0.9866
Epoch 4/5
2719/2719 [==============================] - 2029s 746ms/step - loss: 0.0441 - accuracy: 0.9904
Epoch 5/5
2719/2719 [==============================] - 1834s 674ms/step - loss: 0.0406 - accuracy: 0.9919


In [12]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

1/1 [==============================] - 2s 2s/step - loss: 1.2772e-07 - accuracy: 1.0000
Test accuracy: 1.0


In [13]:
# Save the model
model.save("hand_signal_detection_model.h5")